In [2]:
!pip install pandas numpy yfinance statsmodels scikit-learn
!pip install arch

     -------------------------------------- 84.6/84.6 kB 952.7 kB/s eta 0:00:00
     ---------------------------------------- 64.9/64.9 kB 1.8 MB/s eta 0:00:00
     ------------------------------------ 112.2/112.2 kB 296.3 kB/s eta 0:00:00
     ---------------------------------------- 3.0/3.0 MB 1.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for peewee: filename=peewee-3.17.6-py3-none-any.whl size=138938 sha256=fbba80821559d8a26080164a882f8a081eb8dbec4e737313f5fb3d85827e5423
  Stored in directory: c:\users\prashant\appdata\local\pip\cache\wheels\ee\67\14\8bcb7972056076fb11e959bbda198722e3d0ca73b5fde53f3b
Successfully built peewee
  Attempting uninstall: requests
    Found

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
tensorboard 2.10.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


     -------------------------------------- 925.2/925.2 kB 1.9 MB/s eta 0:00:00


In [19]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model

In [4]:
stock_symbol = 'AAPL'  # Example stock symbol
data = yf.download(stock_symbol, start='2020-01-01', end='2024-08-28')
data = data['Adj Close']  # Use adjusted closing prices
data = data.dropna()

print(data.head())
print(data.tail())

[*********************100%***********************]  1 of 1 completed

Date
2020-01-02    72.876106
2020-01-03    72.167618
2020-01-06    72.742661
2020-01-07    72.400536
2020-01-08    73.565193
Name: Adj Close, dtype: float64
Date
2024-08-21    226.399994
2024-08-22    224.529999
2024-08-23    226.839996
2024-08-26    227.179993
2024-08-27    228.029999
Name: Adj Close, dtype: float64


In [5]:
returns = data.pct_change().dropna()

print(returns.head())

Date
2020-01-03   -0.009722
2020-01-06    0.007968
2020-01-07   -0.004703
2020-01-08    0.016086
2020-01-09    0.021241
Name: Adj Close, dtype: float64


In [6]:
# Fit an ARIMA model to capture mean behavior (optional)
arima_model = ARIMA(returns, order=(1, 0, 0)).fit()

# Fit a GARCH model to capture volatility
garch_model = arch_model(arima_model.resid, vol='Garch', p=1, q=1)
garch_result = garch_model.fit(disp='off')

print(garch_result.summary())

C:\Users\PRASHANT\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\PRASHANT\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\PRASHANT\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                     Constant Mean - GARCH Model Results                      
Dep. Variable:                   None   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                3014.56
Distribution:                  Normal   AIC:                          -6021.12
Method:            Maximum Likelihood   BIC:                          -6000.86
                                        No. Observations:                 1170
Date:                Sat, Aug 31 2024   Df Residuals:                     1169
Time:                        15:13:54   Df Model:                            1
                                  Mean Model                                 
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
mu         6.0684e-04  6.044e-04      1.004      0.315 

C:\Users\PRASHANT\anaconda3\lib\site-packages\arch\univariate\base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0004114. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [7]:
# Forecast future volatility
forecast_horizon = 5  # Number of days to forecast
forecast = garch_result.forecast(horizon=forecast_horizon)
forecast_variance = forecast.variance[-1:]

# Calculate the predicted returns
predicted_volatility = forecast_variance ** 0.5  # Volatility is the square root of variance
print(predicted_volatility)

                 h.1      h.2       h.3       h.4       h.5
Date                                                       
2024-08-27  0.012621  0.01282  0.013011  0.013196  0.013374


In [8]:
# Example simple decision logic based on volatility forecast
threshold = 0.02  # Set a threshold for decision making

if predicted_volatility.iloc[-1, 0] > threshold:
    print("High volatility predicted, consider selling or holding.")
else:
    print("Low volatility predicted, consider buying.")

Low volatility predicted, consider buying.
